In [1]:
from dotenv import load_dotenv
import os
from langchain_teddynote import logging
print(load_dotenv())
print(f"[API KEY]\n{os.environ['OPENAI_API_KEY'][:-15]}" + "*" * 15)
logging.langsmith("01_Basic")

True
[API KEY]
sk-None-FZcW7iODyxjiAh7qYSpIT3BlbkFJyqnfN***************
LangSmith 추적을 시작합니다.
[프로젝트명]
01_Basic


# 데이터를 효과적으로 전달하는 방법

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
prompt = PromptTemplate.from_template("{num} 의 10배는?")
llm = ChatOpenAI(temperature=0)
chain = prompt | llm

In [3]:
chain.invoke({"num":5})

AIMessage(content='50입니다.', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 16, 'total_tokens': 19}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-cf4d0a88-9681-4bb9-97a1-e2eba362d531-0', usage_metadata={'input_tokens': 16, 'output_tokens': 3, 'total_tokens': 19})

In [4]:
chain.invoke(5)

AIMessage(content='50입니다.', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 16, 'total_tokens': 19}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2ceeb083-548c-4048-8717-e416fad38cd9-0', usage_metadata={'input_tokens': 16, 'output_tokens': 3, 'total_tokens': 19})

In [5]:
from langchain_core.runnables import RunnablePassthrough
RunnablePassthrough().invoke({"num": 10})

{'num': 10}

In [6]:
runnable_chain = {"num": RunnablePassthrough()} | prompt | ChatOpenAI()
runnable_chain.invoke(10)

AIMessage(content='100입니다.', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 16, 'total_tokens': 19}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c8f33898-26f7-4db8-b538-61603e39a7d1-0', usage_metadata={'input_tokens': 16, 'output_tokens': 3, 'total_tokens': 19})

In [7]:
RunnablePassthrough().invoke({"num":1})

{'num': 1}

In [8]:
(RunnablePassthrough.assign(new_num = lambda x: x['num']*3)).invoke({"num":1})

{'num': 1, 'new_num': 3}

# RunnableParallel

In [13]:
from langchain_core.runnables import RunnableParallel
runnable = RunnableParallel(
    passed = RunnablePassthrough(),
    extra = RunnablePassthrough.assign(mult= lambda x: x['num']*3),
    modified = lambda x : x["num"] + 1
    )


runnable.invoke({'num':1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

In [14]:
chain1 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country}의 수도는")
    | ChatOpenAI()
)

chain2 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 면적은?")
    | ChatOpenAI()
)

In [15]:
combine_chain = RunnableParallel(capital = chain1,
                                 area = chain2)
combine_chain.invoke('대한민국')

{'capital': AIMessage(content='서울입니다.', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 17, 'total_tokens': 22}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-dc50e130-9b77-406d-b022-5b261cc604a3-0', usage_metadata={'input_tokens': 17, 'output_tokens': 5, 'total_tokens': 22}),
 'area': AIMessage(content='대한민국의 총 면적은 약 100,363 km² 입니다.', response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 20, 'total_tokens': 45}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a256dfeb-88f4-4bd1-96d1-a0a049e58919-0', usage_metadata={'input_tokens': 20, 'output_tokens': 25, 'total_tokens': 45})}

# RunnableLambda

In [20]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from datetime import datetime

def get_today(a):
    return datetime.today().strftime('%b-%d')

get_today(None)

'Jul-29'

In [21]:
from langchain_core.runnables import RunnableLambda

prompt = PromptTemplate.from_template(
    """{today}가 생일인 유명인 {n}명을 나열하세요.
      생년월일을 표기해 주세요."""
)
llm = ChatOpenAI(temperature=0, model_name='gpt-4o')

chain = (
    {"today": RunnableLambda(get_today),
     "n": RunnablePassthrough()}
     |prompt
     | llm
     | StrOutputParser()
)
print(chain.invoke(3))


물론입니다. 7월 29일이 생일인 유명인 세 명을 생년월일과 함께 나열해 드리겠습니다.

1. **베네딕트 컴버배치 (Benedict Cumberbatch)** - 1976년 7월 29일
   - 영국의 배우로, 영화 "셜록" 시리즈와 "닥터 스트레인지"로 유명합니다.

2. **윌리엄 파월 (William Powell)** - 1892년 7월 29일
   - 미국의 배우로, "The Thin Man" 시리즈로 잘 알려져 있습니다.

3. **지젤 번천 (Gisele Bündchen)** - 1980년 7월 29일
   - 브라질 출신의 슈퍼모델로, 세계적으로 유명한 패션 아이콘입니다.

이 외에도 많은 유명인들이 7월 29일에 태어났습니다.


In [23]:
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate

def lenth_func(text):
    return len(text)

def mutiple_length_func(text1, text2):
    return len(text1) * len(text2)

def mult_lenth_func(_dict):
    return mutiple_length_func(_dict["text1"], _dict["text2"])

prompt = ChatPromptTemplate.from_template("{a} + {b}는 무엇인가요?")
model = ChatOpenAI()

# chain1 = prompt | model
chain = (
    {'a':itemgetter('word1') | RunnableLambda(lenth_func),
     'b': {'text1': itemgetter('word1'), "text2": itemgetter('word2')}
     | RunnableLambda(mult_lenth_func),
     }
     | prompt
     | model
)

chain.invoke({"word1": "hello", "word2": "world"})

AIMessage(content='5 + 25 = 30입니다.', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 20, 'total_tokens': 29}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-008237f0-cc89-4319-b911-3c4b0a6a6148-0', usage_metadata={'input_tokens': 20, 'output_tokens': 9, 'total_tokens': 29})